In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
import analyze_utils

save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification'

r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
    save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

getting dir_names...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:02<00:00, 11.23it/s]


In [3]:
top_prompts = (
    r.groupby(['task_name', 'model_cls', 'seed'])
).first()
top_prompts[['prefixes', 'reciprocal_rank', 'prefix_train_loss', 'prefix_train_acc']]

prefixes  \
task_name  model_cls  seed                                                      
ffb_train  autoprompt 1      maybeneshadow Conorfree子e arguably ))Draft So...   
                      2     liftM vaguely feature one feelingWithUnder _: ...   
                      3     ift Forth Sno complicit HKThirdmitting Forth p...   
           iprompt    1      Possible answers: Yes, No, Maybe, Maybe Not, ...   
                      2      Input: The financial instrument used to finan...   
                      3      What is your answer? Response: Maybe (1) Do y...   
imdb_train autoprompt 1      monogQuote "<IncreNo Canaver described how bi...   
                      2      Bring apr No angularzncdn\n\nPos"}]," layers ...   
                      3     ]}Neither cryptoc Nopleted"}," hashes telling ...   
           iprompt    1      Here are a few example Yeses and Nos. Your Ye...   
                      2     If each of the following options says a time r...   
                      3      In the last sentence of the story “The Wendig...   
rt_train   autoprompt 1      188Whether Lub████████\t disag Shiny articula...   
                      2     \t Presbyterian Tommyconsider limp BungYesヤ fa...   
                      3     o vanishedavourQuality====== outweigh FEMAHear...   
           iprompt    1      If you chose No, please explain yourself. —Th...   
                      2      Input:it is as uncompromising as itPrompt: An...   
                      3      Prompt: You Have Selected: Answer: No - The K...   
sst2_train autoprompt 1      rain                location morally suitable...   
                      2      Sovere CHR Argument intrins Keystone Rain No"...   
                      3      Shap inconsistent Conclusion: Hyp Just       ...   
           iprompt    1      Explain your choice. Make sure to indicate th...   
                      2             It Is A Good Film..  It Is A So Bad Film.   
                      3      I'll go first. Input: of its own brilliance  ...   

                            reciprocal_rank  prefix_train_loss  \
task_name  model_cls  seed                                       
ffb_train  autoprompt 1        3.164557e-03          13.105361   
                      2        2.083333e-02          13.601768   
                      3        1.000000e-10          11.885940   
           iprompt    1        9.433962e-03          10.221680   
                      2        2.976190e-03          15.145481   
                      3        2.777778e-02          14.836690   
imdb_train autoprompt 1        5.494505e-03           8.026237   
                      2        1.029866e-03           5.776756   
                      3        1.000000e-10           9.711077   
           iprompt    1        6.666667e-02          10.497448   
                      2        2.500000e-01          13.442577   
                      3        1.111111e-01          12.631762   
rt_train   autoprompt 1        8.547009e-03          10.674474   
                      2        3.610108e-03          10.971385   
                      3        6.250000e-02           9.301171   
           iprompt    1        2.000000e-01          10.344837   
                      2        1.162791e-02          12.111965   
                      3        1.219512e-02           9.587067   
sst2_train autoprompt 1        1.782531e-03           0.804150   
                      2        1.086957e-03           1.901465   
                      3        4.950495e-03           0.478709   
           iprompt    1        2.000000e-01           0.511253   
                      2        1.000000e+00           0.867331   
                      3        1.388889e-02           0.337237   

                            prefix_train_acc  
task_name  model_cls  seed                    
ffb_train  autoprompt 1             0.000000  
                      2             0.000000  
                      3             0.015625  
          

In [4]:
top_prompts.groupby('model_cls').mean()['reciprocal_rank']

model_cls
autoprompt    0.009417
iprompt       0.158806
Name: reciprocal_rank, dtype: float64

In [5]:
top_prompts2 = pd.read_pickle('/home/jxm3/research/prompting/interpretable-autoprompting/results/autoprompt_sentiment/prompts.pkl')

In [6]:
top_prompts2

prefixes  \
task_name    model_cls  seed                                                      
ffb_train    autoprompt 1      questionable Nope McCorm Agility GuiltyQuable...   
                        2      Maybeiago EUR Vimaterasu estab dime blatorget...   
                        3     maybe Yesthe confined introducefineKeanswerwi...   
             genetic    1      Are the following statements true? Answer: Ma...   
                        2     . Answer: MaybeInput:. Answer: MaybeInput:. An...   
                        3      Budapest. Answer: Maybe (1) - The parent comp...   
imdb_train   autoprompt 1      traditionallyムRewPalest high Nowww神\n �/​ vid...   
                        2      Noamphetamine revealed oxidative Yes MDMA pok...   
                        3      hes noises soombat,'' RetrievedDespite No�� s...   
             genetic    1      Slow-paced movies starring the lead role of C...   
                        2      Noam Chomsky is, by definition, a genius, isn...   
                        3      Write a brief essay answering the prompt "Do ...   
rt_train     autoprompt 1     stroazon �HC predicate Cosmic bind?" NoDecl Ca...   
                        2     ecd Courtesy Feldman intuitive intuanguard inf...   
                        3      WinnFarm; does Shroud Pact"]=> NormalConstruc...   
             genetic    1      "a well acted and directed film." Answer: Yes...   
                        2      no, the movie is well-conceived and executed....   
                        3      no doubt this is a slow-paced film. Answer: Y...   
sst2_train   autoprompt 1     aterasuainment ===============================...   
                        2      Givenensional 255103Whetherarch instance Sand...   
             genetic    1      The story line was good. Answer: Yes - A.C. S...   
                        3      good idea, interesting twist, good story.  An...   
tweets_train autoprompt 1     )]. FAQ\n Does refusing Gameahead recon Bluepr...   
                        2      FAQ confusion frequently claims one permit Co...   
                        3     inkeravaritymeta Licenseader observer225 sprit...   
             genetic    1                    @user   Answer: No(4)   https://t.   
                        2      your day today Answer: No Reply: No Answer: N...   
                        3                    sad.wav, sad.mp3 Answer: No (0xFF,   

                              prefix_train_loss  prefix_train_acc  \
task_name    model_cls  seed                                        
ffb_train    autoprompt 1              2.178623          0.500000   
                        2              2.103834          0.812500   
                        3              1.170963          0.937500   
             genetic    1              1.143480          0.937500   
                        2              1.453786          0.750000   
                        3              1.027310          0.875000   
imdb_train   autoprompt 1              8.388127          0.083333   
                        2              6.394076          0.125000   
                        3              7.643039          0.125000   
             genetic    1              9.327049          0.250000   
                        2              9.540214          0.250000   
                        3             10.370430          0.187500   
rt_train     autoprompt 1              0.769902          0.875000   
                        2              0.952305          0.875000   
                        3              0.987308          0.937500   
             genetic    1              0.693669          1.000000   
                        2              0.720310          0.958333   
                        3              0.830908          0.875000   
sst2_train   autoprompt 1              0.650572          0.812500   
                        2              1.152049          0.875000   
             genetic    1              0.904843          0.937

In [7]:
assert r['checkpoint'].unique()[0] == "EleutherAI/gpt-j-6B"

from iprompt import prompt_classification

model = prompt_classification.create_model(r['checkpoint'].unique()[0])

2023-01-22 19:57:32.695270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-22 19:57:32.895561: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-22 19:57:32.933364: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-22 19:57:34.008620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [17]:
## Compute accuracy given correct prompt and save for each task.
import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


data = []
print('calculating accs...')
n_shots = 1
batch_size = 16

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts2.reset_index().iterrows(), total=len(top_prompts)):
    if 'imdb' not in output['task_name']: continue
    if output['model_cls'] == 'autoprompt': continue
    args = argparse.Namespace(**output)
    args.task_name = args.task_name.replace('_train', '_test')
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 1000
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "Does the following input have positive sentiment? " # tmp
    manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=descr, multi_token=False, verbose=True,
        max_length=64, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
    )
    print(output)
    print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=(output['prefixes'] + '. '), multi_token=False, verbose=True,
        max_length=64, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)
    break

calculating accs...


  0%|          | 0/24 [00:00<?, ?it/s]

imdb_test
**loading data: imdb // test


Found cached dataset imdb (/home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-903cc9bdd9c6cd62.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-6f8ce7ac1c3fe85e.arrow
Loading cached processed dataset at /home/jxm3/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-b6aa89e2a4882e14.arrow


  0%|          | 0/16 [00:00<?, ?it/s]

x_text[0]:'Does the following input have positive sentiment? Input: "It\'s really a shame there was so much controversy surrounding this picture (in other words, the infamous affair between Ingrid Bergman and Roberto Rossellini), as this drew a lot of attention to a very forgettable film. The story is incredibly slow, cheap looking and uninteresting. And, to top this off, the film is stuffed to the brim with amateurish actors who can barely speak any English. I really would have preferred subtitles--it would have made the experience less exhausting. Back to the rotten acting. The Neo-realist movement was big in Italy at the time this film was made. Many of these films are absolute masterpieces (such as De Sica\'s Umberto D., The Children Are Watching Us and Miracle in Milan) because they got so much out of the non-professional actors. They behaved like normal people, but in Stromboli they act like normal people TRYING to be actors and come off very badly--like kids in a school play. <b

  0%|          | 0/16 [00:00<?, ?it/s]

x_text[0]:' Slow-paced movies starring the lead role of Chris Hemsworth. Answer: Yes. Input: "It\'s really a shame there was so much controversy surrounding this picture (in other words, the infamous affair between Ingrid Bergman and Roberto Rossellini), as this drew a lot of attention to a very forgettable film. The story is incredibly slow, cheap looking and uninteresting. And, to top this off, the film is stuffed to the brim with amateurish actors who can barely speak any English. I really would have preferred subtitles--it would have made the experience less exhausting. Back to the rotten acting. The Neo-realist movement was big in Italy at the time this film was made. Many of these films are absolute masterpieces (such as De Sica\'s Umberto D., The Children Are Watching Us and Miracle in Milan) because they got so much out of the non-professional actors. They behaved like normal people, but in Stromboli they act like normal people TRYING to be actors and come off very badly--like 